<a href="https://colab.research.google.com/github/BMugo84/pytorch_in_25_hours/blob/main/attn_is_all_you_need_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset tiny Shakespear Dataset
!wget https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt

--2024-09-10 22:28:52--  https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘input.txt’

input.txt               [ <=>                ]   1.42M  --.-KB/s    in 0.09s   

2024-09-10 22:28:52 (16.1 MB/s) - ‘input.txt’ saved [1485051]

